# **Setup**

In [ ]:
import wandb
wandb.login()

In [ ]:
import torch
from torch.utils.data import DataLoader
#from torchsummary import summary
import gc
import h5py
from UNet import UNet, Encoder
from LITSDataset import LITSBinaryDataset
import LossFunctions
import TrainingEval

In [16]:
#Hyperparameters and training modifications
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

modelName = "JointTrainUNet0.5-0.5"
modelFile = "UsedModels/" + modelName
configFile = "Run 2/jointConfigRun2.txt"

startEpoch = 0
useWandB = 0
batchSize = 0
learnRate = 0
epochs = 0
startDim = 0
epochsToDouble = 0
progressive = 0
epochsToSave = 0
cosineAnnealing = 0
cosineRestartEpochs = 0

varDict = {
    "startEpoch":startEpoch,
    "useWandB":useWandB,
    "batchSize":batchSize,
    "learnRate":learnRate,
    "epochs":epochs,
    "startDim":startDim,
    "epochsToDouble":epochsToDouble,
    "progressive":progressive,
    "epochsToSave":epochsToSave,
    "cosineAnnealing":cosineAnnealing,
    "cosineRestartEpochs":cosineRestartEpochs,
}

TrainingEval.ParseConfig(configFile, varDict)

for key in varDict:
    if varDict[key].is_integer():
        locals()[key] = int(varDict[key])
    else:
        locals()[key] = varDict[key]

In [17]:
#Load Datasets
trainDataset = LITSBinaryDataset("Datasets/FullTrainDataset.hdf5")
validationDataset = LITSBinaryDataset("Datasets/ValidationDataset.hdf5")
testDataset = LITSBinaryDataset("Datasets/TestDataset.hdf5")

trainIter = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)
validationIter = DataLoader(validationDataset, batch_size=batchSize)
testIter = DataLoader(testDataset, batch_size=batchSize)

print("Datasets loaded")

Datasets loaded


# **Standard Training**

In [ ]:
#Load model
initModel = ""

lossFuncs = [[LossFunctions.dice_loss, LossFunctions.avgHausdorff], []]
weights = [[1, 0], []]

segmenter = UNet(1)
#print(summary(net, (1, 256, 256)))

#Loads model from file if using a pretrained version
if initModel != "":
    segmenter.load_state_dict(torch.load(initModel))

segmenter = segmenter.to(device)

print("Intialized standard UNet model")

In [ ]:
#Train Model

if useWandB:
    wandb.init(project="LiverSegmentation",
            name=modelName,
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble
            })

TrainingEval.train(segmenter, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=0)

if useWandB:
    wandb.finish()

# **Pre-Training**

In [ ]:
focal = LossFunctions.FocalLoss(weight0=0.2, weight1=0.8, gamma=2)

lossFuncs = [[], [focal]]
weights = [[], [1]]

initEncoder = ""
encoderFile = "UsedModels/ProgressiveEncoder4"

encoder = Encoder(1)

if initEncoder != "":
    encoder.load_state_dict(torch.load(initEncoder))

encoder = encoder.to(device)

In [ ]:
#Train Model
gc.collect()

if useWandB:
    wandb.init(project="PreTrainedEncoder",
            name="UNetEncoder",
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble,
            })

print(TrainingEval.train(encoder, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, encoderFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=progressive, encoder=True))

if useWandB:
    wandb.finish()

In [ ]:
lossFunc = LossFunctions.dice_loss

encoder.load_state_dict(torch.load(encoderFile))

segmenter = UNet(encoder=encoder)
segmenter = segmenter.to(device)

In [ ]:
#Train Model
gc.collect()

if useWandB:
    wandb.init(project="LiverSegmentationPreTraining",
            name="NoWeights",
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble
            })

TrainingEval.train(segmenter, trainIter, validationIter, epochs, startEpoch, learnRate, batchSize, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=0, lossFunc=lossFunc, classification=False)

if useWandB:
    wandb.finish()

# **Joint Training**

In [18]:
segmenter = UNet()

if useWandB:
    wandb.init(project="LiverSegmentationJointTraining",
            name="Weights:",
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble
            })

classLossFunc = LossFunctions.FocalLoss(weight0=0.2, weight1=0.8, gamma=2)

lossFuncs = [[LossFunctions.dice_loss, LossFunctions.dice_score], [classLossFunc]]
weights = [[0.5, 0], [0.5]]

TrainingEval.train(segmenter, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=0)

if useWandB:
    wandb.finish()

Training on cuda
Epoch 0:
Train Loss: 0.42511942982673645 Validation Loss: 0.49436280131340027 dice_loss: tensor(0.9000, device='cuda:0') dice_score: tensor(0.1082, device='cuda:0') FocalLoss: tensor(0.0887, device='cuda:0') 
Epoch 1:
Train Loss: 0.35181134939193726 Validation Loss: 0.4692806303501129 dice_loss: tensor(0.8306, device='cuda:0') dice_score: tensor(0.1747, device='cuda:0') FocalLoss: tensor(0.1080, device='cuda:0') 
Epoch 2:
Train Loss: 0.32451122999191284 Validation Loss: 0.48027461767196655 dice_loss: tensor(0.8857, device='cuda:0') dice_score: tensor(0.1167, device='cuda:0') FocalLoss: tensor(0.0749, device='cuda:0') 
Epoch 3:
Train Loss: 0.3036743402481079 Validation Loss: 0.468526691198349 dice_loss: tensor(0.7465, device='cuda:0') dice_score: tensor(0.4281, device='cuda:0') FocalLoss: tensor(0.1905, device='cuda:0') 
Epoch 4:
Train Loss: 0.28877314925193787 Validation Loss: 0.2742435038089752 dice_loss: tensor(0.5234, device='cuda:0') dice_score: tensor(0.8133, devi

KeyboardInterrupt: 

# **Evaluation/Ending**

In [15]:
modelName = ""
classification = False
modelFile = "UsedModels/JointTrainUNet0.5-0.5BestLoss"
classLossFunc = LossFunctions.FocalLoss(weight0=0.2, weight1=0.8, gamma=2)

lossFuncs = [[LossFunctions.dice_score, LossFunctions.avgHausdorff], [LossFunctions.accuracy, LossFunctions.f1]]

if classification:
    net = encoder
    net.load_state_dict(torch.load(encoderFile))
else:
    net = UNet(1).to(device)
    net.load_state_dict(torch.load(modelFile))

#Evaluate Model
print(f"Model: {modelName}")

losses = TrainingEval.evaluate(net, testIter, lossFuncs, device=device, encoder=False)
logStr = ""
for i, arr in enumerate(losses):
    for j, val in enumerate(arr):
        logStr += (lossFuncs[i][j].__name__ if str(type(lossFuncs[i][j])) == "<class 'function'>" else type(lossFuncs[i][j]).__name__) + ": " + str(val) + " "

print(logStr)

Model: 
dice_score: tensor(0.9203, device='cuda:0') avgHausdorff: 3.4433936719399916 accuracy: 0.9381313131313131 f1: 0.48667628667628654 


In [ ]:
modelFile = ""
dataset = LITSBinaryDataset("")
iter = DataLoader(dataset, batch_size=batchSize)

net = UNet(0)
net.load_state_dict(torch.load(modelFile))

net.to(device)

segmentationMask = TrainingEval.getMasks(net, iter, device=device)

masksFile = ""
wFile = h5py.File(masksFile, "w")

for i, slice in enumerate(segmentationMask):
    wFile.create_dataset("Slice" + str(i), data=slice)

wFile.close()

In [ ]:
#Close datasets
trainDataset.closeFile()
validationDataset.closeFile()
testDataset.closeFile()